In [1]:
import pandas as pd
import json

In [2]:
def del_none(d):
    for key, value in list(d.items()):
        if value is None:
            del d[key]
        elif isinstance(value, dict):
            del_none(value)
    return d

In [3]:
pub_df = pd.read_csv("publications_tab.csv",delimiter="\t")
pub_j = pub_df.to_json(orient="records")

In [4]:
pub_list = []
for obj in json.loads(pub_j):
    pub_list.append(del_none(obj))

In [5]:
for obj in pub_list:
    obj["authors"] = []

In [6]:
rel_pub = pd.read_csv("relationship.csv", delimiter='|')
aut_df = pd.read_csv("author.csv", delimiter='|')

rel_pub = pd.merge(rel_pub, aut_df, left_on="author_name", right_on="name")

In [9]:
work_df = pd.read_csv("work_rel.csv", delimiter='|')
rel_pub = pd.merge(rel_pub, work_df, left_on="author_name", right_on="name")

In [10]:
for index, row in rel_pub.iterrows():
    doi = row["id"]
    for obj in pub_list:
        if obj["id"] == doi:
            author = {}
            author["name"] = row["author_name"]
            author["orcid"] = row["orcid"]
            author["affiliation"] = row["univerity"]
            
            obj["authors"].append(author)

In [11]:
jour_df = pd.read_csv("journals_relationship.csv", delimiter="|")
jour_df = jour_df.dropna()

for index, row in jour_df.iterrows():
    doi = row["id"]
    for obj in pub_list:
        if obj["id"] == doi:
            obj["journal"] = row["name"]

In [12]:
conf_df = pd.read_csv("conferences_relationship.csv", delimiter="|")
conf_df = conf_df.dropna()

for index, row in conf_df.iterrows():
    doi = row["id"]
    for obj in pub_list:
        if obj["id"] == doi:
            obj["conference"] = row["name"]

In [13]:
publisher_df = pd.read_csv("publisher_relationship.csv", delimiter="|")
publisher_df = publisher_df.dropna()

for index, row in publisher_df.iterrows():
    doi = row["id"]
    for obj in pub_list:
        if obj["id"] == doi:
            obj["publisher"] = row["name"]

In [14]:
with open('pub_wip.json', 'w') as f:
    json.dump(pub_list, f, indent=2)